# Tarefa 2: Executar o processamento de dados com o SageMaker Processing

Neste notebook, você vai definir o ambiente necessário para executar uma aplicação básica do Apache Spark usando o Amazon SageMaker Processing. Usando o Apache Spark no SageMaker Processing, você pode executar os trabalhos do Spark sem precisar provisionar um cluster do Amazon EMR. Depois, você vai definir e executar um trabalho do Spark usando a classe **PySparkProcessor** do **SDK Python do SageMaker**. Por fim, você vai validar os resultados do processamento de dados salvos no Amazon Simple Storage Service (Amazon S3).

O script de processamento executa um processamento básico de dados, como a indexação de string, a codificação one-hot, a montagem de vetor e uma divisão 80-20 dos dados processados para treinar e validar conjuntos de dados.     

## Tarefa 2.1: Configuração do ambiente

Instale o pacote mais recente do SDK Python do SageMaker e outras dependências.

In [ ]:
%%capture
%pip install awscli --upgrade
%pip install boto3 --upgrade
%pip install -U "sagemaker>2.0"

1. Depois de atualizar o SDK, reinicie o kernel do notebook selecionando o ícone **Restart kernel** (Reiniciar kernel) na barra de ferramentas do notebook.


Agora, importe as bibliotecas necessárias, faça com que a função de execução execute o trabalho do SageMaker Processing e defina o bucket do Amazon S3 para armazenar as saídas do trabalho do Spark.


In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader
from time import gmtime, strftime

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the Spark processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

Se houver erro, reinicie o kernel do notebook selecionando o ícone **Restart kernel** (Reiniciar kernel) na barra de ferramentas do notebook. Depois, execute a célula novamente.

## Tarefa 2.2: Executar o trabalho de processamento do SageMaker

Nesta tarefa, você vai importar e examinar o conjunto de dados pré-processado.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/spark_adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/spark_adult_data.csv", header=None)
shape.sample(5)

Depois, crie a classe PySparkProcessor do Spark do SageMaker para definir e executar uma aplicação do Spark como um trabalho de processamento. Consulte [PySparkProcessor do Spark do SageMaker](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor) para saber mais sobre essa classe.

Para criar a classe PySparkProcessor, configure os seguintes parâmetros:
- **base_job_name**: prefixo do nome do trabalho de processamento
- **framework_version**: versão do PySpark do SageMaker
- **role**: função de execução do SageMaker
- **instance_count**: número de instâncias para executar o trabalho de processamento
- **instance_type**: tipo de instância do Amazon Elastic Compute Cloud (Amazon EC2) usado para o trabalho de processamento

In [ ]:
#pyspark-processor
from sagemaker.spark.processing import PySparkProcessor

# create a PySparkProcessor
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-preprocessor",
    framework_version="3.1", # Spark version
    role=role,
    instance_count=2,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200
)

Depois, use o método de execução PySparkProcessor para executar o script **pyspark_preprocessing.py** como um trabalho de processamento. Consulte [método de execução PySparkProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor.run) para saber mais sobre esse método. Neste laboratório, as transformações de dados, como indexação de string e codificação one-hot são executadas nos recursos categóricos.

Para executar o trabalho de processamento, configure os seguintes parâmetros:
- **submit_app**: caminho do script de pré-processamento 
- **outputs**: caminho da saída do script de pré-processamento (locais de saída do Amazon S3)
- **arguments**: argumentos de linha de comando para o script de pré-processamento (como os locais de entrada e saída do Amazon S3)

O trabalho de processamento exige cerca de cinco minutos para ser concluído. Enquanto o trabalho estiver em execução, examine o código do script de pré-processamento (que foi pré-configurado como parte deste laboratório) abrindo o arquivo **pyspark_preprocessing.py** com o navegador de arquivos.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
spark_processor.run(
    submit_app='s3://' + os.path.join(bucket, scripts_prefix, "pyspark_preprocessing.py"),
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="validation_data", 
                         source="/opt/ml/processing/validation",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "validation")),
    ],
    arguments=[
        "--s3_input_bucket", bucket,
        "--s3_input_key_prefix", input_raw_data_prefix,
        "--s3_output_bucket", bucket,
        "--s3_output_key_prefix", output_preprocessed_data_prefix],
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, logs_prefix),
    logs=True
)

print("Spark Processing Job Completed.")

## Tarefa 2.3: Validar os resultados do processamento de dados

Valide a saída do trabalho de processamento de dados que você executou examinando as cinco primeiras linhas dos conjuntos de dados de saída de treinamento e teste.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/validation/validation_features.csv - | head -n5

### Conclusão

Parabéns! Você usou o SageMaker Processing para criar um trabalho de processamento do Spark usando o SDK Python do SageMaker e executou um trabalho de processamento com sucesso.

A próxima tarefa do laboratório se concentra no processamento de dados com o SageMaker Processing e o contêiner scikit-learn integrado.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue na **Tarefa 3: Executar o processamento de dados com o SageMaker Processing e o contêiner scikit-learn integrado**.